In [1]:
from __future__ import print_function
import sys
sys.setrecursionlimit(99999)
import pdb

import numpy as np
np.random.seed(1337)  # for reproducibility

import keras

from keras.datasets import mnist, cifar10
from keras.models import Sequential, Graph
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import ZeroPadding2D, AveragePooling2D, Convolution2D
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint
from keras.layers.normalization import BatchNormalization

import residual_blocks


batch_size = 2
nb_classes = 1
nb_epoch = 200

def compute_padding_length(length_before, stride, length_conv):
    ''' Assumption: you want the subsampled result has a length of floor(original_length/stride).
    '''
    N = length_before
    F = length_conv
    S = stride
    if S == F:
        return 0
    if S == 1:
        return (F-1)/2
    for P in range(S):
        if (N-F+2*P)/S + 1 == N/S:
            return P
    return 0

def design_for_residual_blocks(num_channel_input=1):
    ''''''
    model = Sequential() # it's a CONTAINER, not MODEL
    # set numbers
#     num_big_blocks = 6
#     image_patch_sizes = [[6,6],[6,6],[9,9],[9,9],[9,9],[9,9]]
#     pool_sizes = [(2,2)]*num_big_blocks
#     n_features = [24, 24, 40, 40, 288, 288]
#     n_features_next = [24, 40, 40, 288, 288,288]
#     height_input = 144
#     width_input = 144
    num_big_blocks = 21
    image_patch_sizes = [[3,3]]*num_big_blocks
    pool_sizes = [(2,2)]*num_big_blocks
    n_features =      [24,24,24,24,48,48,48,64,96,96,96, 128,128,128,192,192,192,192,384,384,384]
    n_features_next = [24,24,24,48,48,64,48,96,96,96,128,128,128,192,192,192,192,384,384,384,384]
    height_input = 144
    width_input = 144
    
    for conv_idx in range(num_big_blocks):    
        n_feat_here = n_features[conv_idx]
        # residual block 0
        model.add(residual_blocks.building_residual_block(  (num_channel_input, height_input, width_input),
                                                            n_feat_here,
                                                            kernel_sizes=image_patch_sizes[conv_idx]
                                                            ))

        # residual block 1 (you can add it as you want (and your resources allow..))
        if False:
            model.add(residual_blocks.building_residual_block(  (n_feat_here, height_input, width_input),
                                                                n_feat_here,
                                                                kernel_sizes=image_patch_sizes[conv_idx]
                                                                ))
        
        # the last residual block N-1
        # the last one : pad zeros, subsamples, and increase #channels
        pad_height = compute_padding_length(height_input, pool_sizes[conv_idx][0], image_patch_sizes[conv_idx][0])
        pad_width = compute_padding_length(width_input, pool_sizes[conv_idx][1], image_patch_sizes[conv_idx][1])

        model.add(ZeroPadding2D(padding=(pad_height,pad_width))) 
        height_input += 2*pad_height
        width_input += 2*pad_width
        n_feat_next = n_features_next[conv_idx]
        if n_features[conv_idx]!=n_features_next[conv_idx]:
            model.add(residual_blocks.building_residual_block(  (n_feat_here, height_input, width_input),
                                                                n_feat_next,
                                                                kernel_sizes=image_patch_sizes[conv_idx],
                                                                is_subsample=True,
                                                                subsample=pool_sizes[conv_idx]
                                                                ))

        height_input, width_input = model.output_shape[2:]
        # width_input  = int(width_input/pool_sizes[conv_idx][1])
        num_channel_input = n_feat_next

    # Add average pooling at the end:
    print('Average pooling, from (%d,%d) to (1,1)' % (height_input, width_input))
    model.add(AveragePooling2D(pool_size=(height_input, width_input)))

    return model

def get_residual_model(is_mnist=True, img_channels=1, img_rows=144, img_cols=144):
    model = keras.models.Sequential()
    first_layer_channel = 24
    if is_mnist: # size to be changed to 32,32
        model.add(ZeroPadding2D((0,0), input_shape=(img_channels, img_rows, img_cols))) # resize (28,28)-->(32,32)
        # the first conv 
        model.add(Convolution2D(first_layer_channel, 3, 3, border_mode='same'))
    else:
        model.add(Convolution2D(first_layer_channel, 3, 3, border_mode='same', input_shape=(img_channels, img_rows, img_cols)))

    model.add(Activation('relu'))
    # [residual-based Conv layers]
    residual_blocks = design_for_residual_blocks(num_channel_input=first_layer_channel)
    model.add(residual_blocks)
    model.add(BatchNormalization(axis=1))
    model.add(Activation('relu'))
    # [Classifier]    
    model.add(Flatten())
    model.add(Dense(nb_classes))
    model.add(Activation('linear'))
    # [END]
    return model

def exp_error(y_true, y_pred):
    return K.mean(K.square(np.exp(y_pred) - np.exp(y_true)))


if __name__ =='__main__':
    
    is_mnist = True
    is_cifar10 = not is_mnist
    
    if is_mnist:
        import scipy.io as sio
        import numpy as np
        img_rows, img_cols = 144, 144
        img_channels = 1
        WB=sio.loadmat('WB_small.mat')['WB_small'].T
        Y_data=sio.loadmat('true.mat')['skt']
        Y_data=(Y_data-min(Y_data))/(max(Y_data)-min(Y_data))
        X_data=np.reshape(WB,(100,1,img_rows,img_cols))
        X_train=X_data[0:80];X_test=X_data[80:100]
        Y_train=Y_data[0:80];Y_test=Y_data[80:100]
        print(' == MNIST ==')
#         (X_train, y_train), (X_test, y_test) = mnist.load_data()
#         img_rows, img_cols = 28, 28
#         img_channels = 1
#         print(' == MNIST ==')

    X_train = X_train.reshape(X_train.shape[0], img_channels, img_rows, img_cols)
    X_test = X_test.reshape(X_test.shape[0], img_channels, img_rows, img_cols)
    X_train = X_train.astype('float32')
    X_test = X_test.astype('float32')
    # X_train /= 255
    # X_test /= 255
    #X_train = (X_train - np.mean(X_train))/np.std(X_train)
    #X_test = (X_test - np.mean(X_test))/np.std(X_test)
    
#     X_train = (X_train - np.mean(X_train))/np.std(X_train)
#     X_test = (X_test - np.mean(X_test))/np.std(X_test)
#     Y_train = y_train
#     Y_test = y_test
    print('X_train shape:', X_train.shape)
    print(X_train.shape[0], 'train samples')
    print(X_test.shape[0], 'test samples')

    model = get_residual_model(is_mnist=is_mnist, img_channels=img_channels, img_rows=img_rows, img_cols=img_cols)

    model.compile(loss='mae', optimizer='adam', metrics=['accuracy'])

#     # autosave best Model
#     best_model_file = "./my_model_weights.h5"
#     best_model = ModelCheckpoint(best_model_file, verbose=1, save_best_only=True)
    
#     model.fit(X_train, Y_train, batch_size=batch_size, nb_epoch=nb_epoch,
#               verbose=1, validation_data=(X_test, Y_test))#, callbacks=[best_model])
#     score = model.evaluate(X_test, Y_test, show_accuracy=True, verbose=0)


Using Theano backend.
DEBUG: nvcc STDOUT mod.cu
   Creating library C:/Users/p2admin/AppData/Local/Theano/compiledir_Windows-7-6.1.7601-SP1-Intel64_Family_6_Model_62_Stepping_4_GenuineIntel-2.7.12-64/tmptckrru/265abc51f7c376c224983485238ff1a5.lib and object C:/Users/p2admin/AppData/Local/Theano/compiledir_Windows-7-6.1.7601-SP1-Intel64_Family_6_Model_62_Stepping_4_GenuineIntel-2.7.12-64/tmptckrru/265abc51f7c376c224983485238ff1a5.exp

Using gpu device 0: Quadro K2000 (CNMeM is disabled, cuDNN 5005)


 == MNIST ==
X_train shape: (80L, 1L, 144L, 144L)
80 train samples
20 test samples
   - New residual block with
('      input shape:', (24, 144, 144))
('      kernel size:', [3, 3])
        -- model was built.
   - New residual block with
('      input shape:', (24, 146, 146))
('      kernel size:', [3, 3])
        -- model was built.
   - New residual block with
('      input shape:', (24, 148, 148))
('      kernel size:', [3, 3])
        -- model was built.
   - New residual block with
('      input shape:', (24, 150, 150))
('      kernel size:', [3, 3])
        -- model was built.
   - New residual block with
('      input shape:', (24, 152, 152))
('      kernel size:', [3, 3])
      - Input channels: 24 ---> num feature maps on out: 48
('      - with subsample:', (2, 2))
        -- model was built.
   - New residual block with
('      input shape:', (48, 76, 76))
('      kernel size:', [3, 3])
        -- model was built.
   - New residual block with
('      input shape:', (48, 78, 

In [4]:
model.fit(X_train, Y_train, batch_size=batch_size, nb_epoch=500,verbose=1, validation_data=(X_test, Y_test))

Train on 80 samples, validate on 20 samples
Epoch 1/500
80/80 [==============================] - 126s - loss: 0.1136 - acc: 0.0125 - val_loss: 0.1600 - val_acc: 0.0000e+00
Epoch 2/500
80/80 [==============================] - 129s - loss: 0.1210 - acc: 0.0125 - val_loss: 0.1603 - val_acc: 0.0000e+00
Epoch 3/500
80/80 [==============================] - 128s - loss: 0.1199 - acc: 0.0125 - val_loss: 0.1628 - val_acc: 0.0000e+00
Epoch 4/500
80/80 [==============================] - 126s - loss: 0.1234 - acc: 0.0125 - val_loss: 0.1615 - val_acc: 0.0000e+00
Epoch 5/500
80/80 [==============================] - 851s - loss: 0.1180 - acc: 0.0125 - val_loss: 0.1639 - val_acc: 0.0000e+00
Epoch 6/500
80/80 [==============================] - 1047s - loss: 0.1183 - acc: 0.0125 - val_loss: 0.1587 - val_acc: 0.0000e+00
Epoch 7/500
80/80 [==============================] - 590s - loss: 0.1194 - acc: 0.0125 - val_loss: 0.1630 - val_acc: 0.0000e+00
Epoch 8/500
80/80 [==============================] - 131s -

In [ ]:
keras.optimizers.Adam(lr=0.00001, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
model.compile(loss='mae', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, Y_train, batch_size=batch_size, nb_epoch=500,verbose=1, validation_data=(X_test, Y_test))

In [10]:
Y_test-model.predict(X_test)

array([[ 0.0598288 ],
       [-0.22975894],
       [ 0.02204274],
       [ 0.16350582],
       [-0.26330754],
       [-0.18016099],
       [-0.46485456],
       [-0.32365561],
       [ 0.09812948],
       [ 0.10863168],
       [-0.24898307],
       [-0.01799552],
       [ 0.07780188],
       [-0.09783778],
       [-0.14135144],
       [ 0.2093093 ],
       [-0.05261862],
       [ 0.20715234],
       [ 0.10180938],
       [ 0.39897717]])

In [ ]:
model = get_residual_model(is_mnist=is_mnist, img_channels=img_channels, img_rows=img_rows, img_cols=img_cols)

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# autosave best Model
best_model_file = "./my_model_weights.h5"
best_model = ModelCheckpoint(best_model_file, verbose=1, save_best_only=True)

model.fit(X_train, Y_train, batch_size=batch_size, nb_epoch=nb_epoch,
          verbose=1, validation_data=(X_test, Y_test))#, callbacks=[best_model])
score = model.evaluate(X_test, Y_test, show_accuracy=True, verbose=0)
print('Test score:', score[0])
print('Test accuracy:', score[1])

In [3]:
model.evaluate(X_test, Y_test, show_accuracy=True, verbose=0)

[0.14737975597381592, 0.0]

In [2]:
'''Example of using residual_blocks.py by Keunwoo Choi (keunwoo.choi@qmul.ac.uk), on Keras 0.3.2
It's copy-and-pasted from the code I am using, so it wouldn't run.
Just take a look to understand how to use residual blocks. 

The whole structure is...

   -------Residual Model (keras.models.Sequential())-------
   |                                                      |
   |     ---- Residual blocks ------------------------|   |
   |     |    (keras.layers.containers.Sequential())  |   |
   |     |                                            |   |
   |     |     -- Many Residual blocks ------------   |   |
   |     |     | (keras.layers.containers.Graph())|   |   |
   |     |     |                                  |   |   |
   |     |     |__________________________________|   |   |
   |     |____________________________________________|   |
   |                                                      |
   |______________________________________________________|

'''
from __future__ import print_function
import sys
sys.setrecursionlimit(99999)
import pdb

import numpy as np
np.random.seed(1337)  # for reproducibility

import keras

from keras.datasets import mnist, cifar10
from keras.models import Sequential, Graph
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import ZeroPadding2D, AveragePooling2D, Convolution2D
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint
from keras.layers.normalization import BatchNormalization

import residual_blocks


batch_size = 10
nb_classes = 1
nb_epoch = 20

def compute_padding_length(length_before, stride, length_conv):
    ''' Assumption: you want the subsampled result has a length of floor(original_length/stride).
    '''
    N = length_before
    F = length_conv
    S = stride
    if S == F:
        return 0
    if S == 1:
        return (F-1)/2
    for P in range(S):
        if (N-F+2*P)/S + 1 == N/S:
            return P
    return None

def design_for_residual_blocks(num_channel_input=1):
    ''''''
    model = Sequential() # it's a CONTAINER, not MODEL
    # set numbers
    num_big_blocks = 5
    image_patch_sizes = [[3,3]]*num_big_blocks
    pool_sizes = [(2,2)]*num_big_blocks
    n_features =      [128, 256, 512, 512, 1024]
    n_features_next = [256, 512, 512, 512, 1024]
    height_input = 148
    width_input = 148
    for conv_idx in range(num_big_blocks):    
        n_feat_here = n_features[conv_idx]
        # residual block 0
        model.add(residual_blocks.building_residual_block(  (num_channel_input, height_input, width_input),
                                                            n_feat_here,
                                                            kernel_sizes=image_patch_sizes[conv_idx]
                                                            ))

        # residual block 1 (you can add it as you want (and your resources allow..))
        if False:
            model.add(residual_blocks.building_residual_block(  (n_feat_here, height_input, width_input),
                                                                n_feat_here,
                                                                kernel_sizes=image_patch_sizes[conv_idx]
                                                                ))
        
        # the last residual block N-1
        # the last one : pad zeros, subsamples, and increase #channels
        pad_height = compute_padding_length(height_input, pool_sizes[conv_idx][0], image_patch_sizes[conv_idx][0])
        pad_width = compute_padding_length(width_input, pool_sizes[conv_idx][1], image_patch_sizes[conv_idx][1])
        model.add(ZeroPadding2D(padding=(pad_height,pad_width))) 
        height_input += 2*pad_height
        width_input += 2*pad_width
        n_feat_next = n_features_next[conv_idx]
        model.add(residual_blocks.building_residual_block(  (n_feat_here, height_input, width_input),
                                                            n_feat_next,
                                                            kernel_sizes=image_patch_sizes[conv_idx],
                                                            is_subsample=True,
                                                            subsample=pool_sizes[conv_idx]
                                                            ))

        height_input, width_input = model.output_shape[2:]
        # width_input  = int(width_input/pool_sizes[conv_idx][1])
        num_channel_input = n_feat_next

    # Add average pooling at the end:
    print('Average pooling, from (%d,%d) to (1,1)' % (height_input, width_input))
    model.add(AveragePooling2D(pool_size=(height_input, width_input)))

    return model

def get_residual_model(is_mnist=True, img_channels=1, img_rows=28, img_cols=28):
    model = keras.models.Sequential()
    first_layer_channel = 128
    if is_mnist: # size to be changed to 32,32
        model.add(ZeroPadding2D((2,2), input_shape=(img_channels, img_rows, img_cols))) # resize (28,28)-->(32,32)
        # the first conv 
        model.add(Convolution2D(first_layer_channel, 3, 3, border_mode='same'))
    else:
        model.add(Convolution2D(first_layer_channel, 3, 3, border_mode='same', input_shape=(img_channels, img_rows, img_cols)))

    model.add(Activation('relu'))
    # [residual-based Conv layers]
    residual_blocks = design_for_residual_blocks(num_channel_input=first_layer_channel)
    model.add(residual_blocks)
    model.add(BatchNormalization(axis=1))
    model.add(Activation('relu'))
    # [Classifier]    
    model.add(Flatten())
    model.add(Dense(nb_classes))
    model.add(Activation('softmax'))
    # [END]
    return model

if __name__ =='__main__':

    is_mnist = True
    is_cifar10 = not is_mnist

    if is_mnist:
        import scipy.io as sio
        import numpy as np

        img_rows, img_cols = 144, 144
        img_channels = 1
        WB = sio.loadmat('WB_small.mat')['WB_small']
        Y_data = sio.loadmat('true.mat')['skt']
        Y_data = (Y_data - min(Y_data)) / (max(Y_data) - min(Y_data))
        X_data = np.reshape(WB, (100, 1, img_rows, img_cols))
        X_train = X_data[0:80];
        X_test = X_data[80:100]
        Y_train = Y_data[0:80];
        Y_test = Y_data[80:100]
        print(' == MNIST ==')
    else:
        (X_train, y_train), (X_test, y_test) = cifar10.load_data()
        img_rows, img_cols = 32, 32
        img_channels = 3
        print(' == CIFAR10 ==')

    X_train = X_train.reshape(X_train.shape[0], img_channels, img_rows, img_cols)
    X_test = X_test.reshape(X_test.shape[0], img_channels, img_rows, img_cols)
    X_train = X_train.astype('float32')
    X_test = X_test.astype('float32')
    # X_train /= 255
    # X_test /= 255
    # X_train = (X_train - np.mean(X_train))/np.std(X_train)
    # X_test = (X_test - np.mean(X_test))/np.std(X_test)
    print('X_train shape:', X_train.shape)
    print(X_train.shape[0], 'train samples')
    print(X_test.shape[0], 'test samples')

    model = get_residual_model(is_mnist=is_mnist, img_channels=img_channels, img_rows=img_rows, img_cols=img_cols)

    model.compile(loss='mse', optimizer='adam', metrics=['accuracy'])

    # # autosave best Model
    # best_model_file = "./my_model_weights.h5"
    # best_model = ModelCheckpoint(best_model_file, verbose=1, save_best_only=True)
    #
#     model.fit(X_train, Y_train, batch_size=batch_size, nb_epoch=nb_epoch, verbose=1, validation_data=(X_test, Y_test))#, callbacks=[best_model])
    # score = model.evaluate(X_test, Y_test, show_accuracy=True, verbose=0)
    # print('Test score:', score[0])
    # print('Test accuracy:', score[1])



 == MNIST ==
X_train shape: (80L, 1L, 144L, 144L)
80 train samples
20 test samples
   - New residual block with
('      input shape:', (128, 148, 148))
('      kernel size:', [3, 3])
        -- model was built.
   - New residual block with
('      input shape:', (128, 150, 150))
('      kernel size:', [3, 3])
      - Input channels: 128 ---> num feature maps on out: 256
('      - with subsample:', (2, 2))
        -- model was built.
   - New residual block with
('      input shape:', (256, 74, 74))
('      kernel size:', [3, 3])
        -- model was built.
   - New residual block with
('      input shape:', (256, 76, 76))
('      kernel size:', [3, 3])
      - Input channels: 256 ---> num feature maps on out: 512
('      - with subsample:', (2, 2))
        -- model was built.
   - New residual block with
('      input shape:', (512, 37, 37))
('      kernel size:', [3, 3])
        -- model was built.
   - New residual block with
('      input shape:', (512, 37, 37))
('      kernel size:

In [3]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
zeropadding2d_5 (ZeroPadding2D)  (None, 1, 148, 148)   0           zeropadding2d_input_2[0][0]      
____________________________________________________________________________________________________
convolution2d_17 (Convolution2D) (None, 128, 148, 148) 1280        zeropadding2d_5[0][0]            
____________________________________________________________________________________________________
activation_16 (Activation)       (None, 128, 148, 148) 0           convolution2d_17[0][0]           
____________________________________________________________________________________________________
sequential_4 (Sequential)        (None, 1024, 1, 1)    73990912    activation_16[0][0]              
___________________________________________________________________________________________

In [9]:
'''Example of using residual_blocks.py by Keunwoo Choi (keunwoo.choi@qmul.ac.uk), on Keras 0.3.2
It's copy-and-pasted from the code I am using, so it wouldn't run.
Just take a look to understand how to use residual blocks. 

The whole structure is...

   -------Residual Model (keras.models.Sequential())-------
   |                                                      |
   |     ---- Residual blocks ------------------------|   |
   |     |    (keras.layers.containers.Sequential())  |   |
   |     |                                            |   |
   |     |     -- Many Residual blocks ------------   |   |
   |     |     | (keras.layers.containers.Graph())|   |   |
   |     |     |                                  |   |   |
   |     |     |__________________________________|   |   |
   |     |____________________________________________|   |
   |                                                      |
   |______________________________________________________|

'''
from __future__ import print_function
import sys
sys.setrecursionlimit(99999)
import pdb

import numpy as np
np.random.seed(1337)  # for reproducibility

import keras

from keras.datasets import mnist, cifar10
from keras.models import Sequential, Graph
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import ZeroPadding2D, AveragePooling2D, Convolution2D
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint
from keras.layers.normalization import BatchNormalization

import residual_blocks


batch_size = 10
nb_classes = 1
nb_epoch = 20

def compute_padding_length(length_before, stride, length_conv):
    ''' Assumption: you want the subsampled result has a length of floor(original_length/stride).
    '''
    N = length_before
    F = length_conv
    S = stride
    if S == F:
        return 0
    if S == 1:
        return (F-1)/2
    for P in range(S):
        if (N-F+2*P)/S + 1 == N/S:
            return P
    return None

def design_for_residual_blocks(num_channel_input=1):
    ''''''
    model = Sequential() # it's a CONTAINER, not MODEL
    # set numbers
    num_big_blocks = 5
    image_patch_sizes = [[3,3]]*num_big_blocks
    pool_sizes = [(2,2)]*num_big_blocks
    n_features =      [128, 256, 512, 512, 1024]
    n_features_next = [256, 512, 512, 512, 1024]
    height_input = 148
    width_input = 148
    for conv_idx in range(num_big_blocks):    
        n_feat_here = n_features[conv_idx]
        # residual block 0
        model.add(residual_blocks.building_residual_block(  (num_channel_input, height_input, width_input),
                                                            n_feat_here,
                                                            kernel_sizes=image_patch_sizes[conv_idx]
                                                            ))

        # residual block 1 (you can add it as you want (and your resources allow..))
        if False:
            model.add(residual_blocks.building_residual_block(  (n_feat_here, height_input, width_input),
                                                                n_feat_here,
                                                                kernel_sizes=image_patch_sizes[conv_idx]
                                                                ))
        
        # the last residual block N-1
        # the last one : pad zeros, subsamples, and increase #channels
        pad_height = compute_padding_length(height_input, pool_sizes[conv_idx][0], image_patch_sizes[conv_idx][0])
        pad_width = compute_padding_length(width_input, pool_sizes[conv_idx][1], image_patch_sizes[conv_idx][1])
        model.add(ZeroPadding2D(padding=(pad_height,pad_width))) 
        height_input += 2*pad_height
        width_input += 2*pad_width
        n_feat_next = n_features_next[conv_idx]
        model.add(residual_blocks.building_residual_block(  (n_feat_here, height_input, width_input),
                                                            n_feat_next,
                                                            kernel_sizes=image_patch_sizes[conv_idx],
                                                            is_subsample=True,
                                                            subsample=pool_sizes[conv_idx]
                                                            ))

        height_input, width_input = model.output_shape[2:]
        # width_input  = int(width_input/pool_sizes[conv_idx][1])
        num_channel_input = n_feat_next

    # Add average pooling at the end:
    print('Average pooling, from (%d,%d) to (1,1)' % (height_input, width_input))
    model.add(AveragePooling2D(pool_size=(height_input, width_input)))

    return model

def get_residual_model(is_mnist=True, img_channels=1, img_rows=28, img_cols=28):
    model = keras.models.Sequential()
    first_layer_channel = 128
    if is_mnist: # size to be changed to 32,32
        model.add(ZeroPadding2D((2,2), input_shape=(img_channels, img_rows, img_cols))) # resize (28,28)-->(32,32)
        # the first conv 
        model.add(Convolution2D(first_layer_channel, 3, 3, border_mode='same'))
    else:
        model.add(Convolution2D(first_layer_channel, 3, 3, border_mode='same', input_shape=(img_channels, img_rows, img_cols)))

    model.add(Activation('relu'))
    # [residual-based Conv layers]
    residual_blocks = design_for_residual_blocks(num_channel_input=first_layer_channel)
    model.add(residual_blocks)
    model.add(BatchNormalization(axis=1))
    model.add(Activation('relu'))
    # [Classifier]    
    model.add(Flatten())
    model.add(Dense(nb_classes))
    model.add(Activation('softmax'))
    # [END]
    return model

if __name__ =='__main__':

    is_mnist = True
    is_cifar10 = not is_mnist

    if is_mnist:
        import scipy.io as sio
        import numpy as np

        img_rows, img_cols = 144, 144
        img_channels = 1
        WB = sio.loadmat('WB_small.mat')['WB_small']
        Y_data = sio.loadmat('true.mat')['skt']
        Y_data = (Y_data - min(Y_data)) / (max(Y_data) - min(Y_data))
        X_data = np.reshape(WB, (100, 1, img_rows, img_cols))
        X_train = X_data[0:80];
        X_test = X_data[80:100]
        Y_train = Y_data[0:80];
        Y_test = Y_data[80:100]
        print(' == MNIST ==')
    else:
        (X_train, y_train), (X_test, y_test) = cifar10.load_data()
        img_rows, img_cols = 32, 32
        img_channels = 3
        print(' == CIFAR10 ==')

    X_train = X_train.reshape(X_train.shape[0], img_channels, img_rows, img_cols)
    X_test = X_test.reshape(X_test.shape[0], img_channels, img_rows, img_cols)
    X_train = X_train.astype('float32')
    X_test = X_test.astype('float32')
    # X_train /= 255
    # X_test /= 255
    # X_train = (X_train - np.mean(X_train))/np.std(X_train)
    # X_test = (X_test - np.mean(X_test))/np.std(X_test)
    print('X_train shape:', X_train.shape)
    print(X_train.shape[0], 'train samples')
    print(X_test.shape[0], 'test samples')

    model = get_residual_model(is_mnist=is_mnist, img_channels=img_channels, img_rows=img_rows, img_cols=img_cols)

    model.compile(loss='mse', optimizer='adam', metrics=['accuracy'])

    # # autosave best Model
    # best_model_file = "./my_model_weights.h5"
    # best_model = ModelCheckpoint(best_model_file, verbose=1, save_best_only=True)
    #
#     model.fit(X_train, Y_train, batch_size=batch_size, nb_epoch=nb_epoch, verbose=1, validation_data=(X_test, Y_test))#, callbacks=[best_model])
    # score = model.evaluate(X_test, Y_test, show_accuracy=True, verbose=0)
    # print('Test score:', score[0])
    # print('Test accuracy:', score[1])



 == MNIST ==
X_train shape: (80L, 1L, 144L, 144L)
80 train samples
20 test samples
   - New residual block with
('      input shape:', (128, 148, 148))
('      kernel size:', [3, 3])
        -- model was built.
   - New residual block with
('      input shape:', (128, 150, 150))
('      kernel size:', [3, 3])
      - Input channels: 128 ---> num feature maps on out: 256
('      - with subsample:', (2, 2))
        -- model was built.
   - New residual block with
('      input shape:', (256, 74, 74))
('      kernel size:', [3, 3])
        -- model was built.
   - New residual block with
('      input shape:', (256, 76, 76))
('      kernel size:', [3, 3])
      - Input channels: 256 ---> num feature maps on out: 512
('      - with subsample:', (2, 2))
        -- model was built.
   - New residual block with
('      input shape:', (512, 37, 37))
('      kernel size:', [3, 3])
        -- model was built.
   - New residual block with
('      input shape:', (512, 37, 37))
('      kernel size: